In [1]:
import pickle
import os

In [ ]:
!pip install -r ../requirements.txt
!pip install -e ..


In [3]:
import protorubric

In [7]:
from protorubric.eval.rubric_with_answers import RubricWithAnswers
from protorubric.constants import EVAL_BASE_DIR

fnames = [path for path in os.listdir(EVAL_BASE_DIR) if path.endswith(".pkl")]

completed_rubrics: dict[str, RubricWithAnswers] = dict()
for fname in fnames:
    completed_rubrics[fname.replace(".pkl", "")] = RubricWithAnswers.load_pkl(os.path.join(EVAL_BASE_DIR, fname))

In [ ]:
completed_rubrics.keys()

In [16]:
metrics = {k: v._results_metrics['metrics'] for k, v in completed_rubrics.items()}
requirements = {k: v._results_metrics['requirements'] for k, v in completed_rubrics.items()}


In [66]:
import pandas as pd

def results_dict_to_structured_df(results_dict: dict[str, dict[str, dict[str, float]]], new_col_name: str, old_key: str = "stats") -> pd.DataFrame:
    df = pd.DataFrame(results_dict)
    df_reset = df.reset_index().melt(id_vars='index', var_name='result_title', value_name=old_key)
    df_reset = df_reset.rename(columns={'index': new_col_name})
    stats_df = pd.json_normalize(df_reset[old_key])
    results_df = pd.concat([df_reset[['result_title', new_col_name]], stats_df], axis=1)
    return results_df

In [ ]:
metrics_df = results_dict_to_structured_df(metrics, "metric")
metrics_df.head()

In [ ]:
results_df = results_dict_to_structured_df(requirements, "requirement")
results_df.head()